## Imports

In [5]:
## resolver problema no caminho do maturin do pyo3, que faz a biblioteca em rust ser acessível em python
import sys, os
for cam in sys.path:
    if cam.find('LSTM') != -1:
        caminho = os.path.join(cam, 'rust_funcs', '.env', 'Lib', 'site-packages', 'rust_funcs')
        print(caminho)
        break
sys.path.append(caminho)
import rust_funcs
sys.path.remove(caminho)
del caminho

import numpy as np

c:\Users\brunO\AreadeTrabalho\Prog\Python\LSTM\rust_funcs\.env\Lib\site-packages\rust_funcs


In [6]:
def relu(x):
    return (x > 0)*x

def tanh(x):
    return np.tanh(x)

def sigmoid(x):
    return (tanh(x/2)+1)/2

def logloss(y, yhat):
    return rust_funcs.logloss(y, yhat)

def crossentropy(y,yhat):
    return rust_funcs.crossentropy(y,yhat)

def quaderror(x, y, f = None):
    if f:
        return np.dot(f(x) - y, f(x) - y)/(2*len(y))
    return rust_funcs.quaderror(x,y)

In [33]:
## teste de tempo entre funções (principalmente para o Rust)

def fazer_fig(listas, listas_std, lista_f):
    import seaborn as sns, matplotlib.pyplot as plt
    fig, axe = plt.subplots(figsize=(14,7), facecolor='lightgray')
    axe.set_facecolor('lightgray')
    for i in range(len(listas)):
        sns.lineplot(data=listas[i], ax=axe, linewidth=0.4)
    for i in range(len(listas)):
        lower_bound = [M_new - Sigma for M_new, Sigma in zip(listas[i], listas_std[i])]
        upper_bound = [M_new + Sigma for M_new, Sigma in zip(listas[i], listas_std[i])]
        plt.fill_between(list(range(7)), lower_bound, upper_bound, alpha=.3, color='red')
    plt.yscale('log')
    axe.legend([lista_f[i//2].__name__ if not i%2 else lista_f[i//2].__name__+'±std' for i in range(0,2*len(lista_f))])
    axe.set_xlabel('Ordem de grandeza do teste')
    axe.set_ylabel('(Log da) Duração do teste')
    axe.set_xticklabels(list(range(0,8)))
    plt.show()

def single_test(n, f, lista_f, y, yhat):
    assert(np.all(np.isclose([[f(y, yhat) for f in lista_f]], lista_f[0](y,yhat))))
    number = %timeit -o f(y, yhat)
    return number

def testar(lista_f):
    k = len(lista_f)
    listas = [[] for i in range(k)]
    for n in range(1, 8):
        y = (np.random.random(10**n-1)+0.5)//1
        yhat = np.linspace(0.05,0.95, 10**n-1)
        for i in range(k):
            listas[i].append(single_test(n, lista_f[i], lista_f, y, yhat))
    listas_std = [[] for i in range(k)]
    for j in range(len(listas[0])):
        for i in range(k):
            listas_std[i].append(listas[i][j].stdev)
            listas[i][j] = listas[i][j].average 
    fazer_fig(listas, listas_std, lista_f)
    
testar([rust_funcs.crossentropy, rust_funcs.crossentropy_teste])
   

829 ns ± 120 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
718 ns ± 39.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
2.42 µs ± 235 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.55 µs ± 464 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
17.8 µs ± 749 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
16.1 µs ± 320 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
185 µs ± 24.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
229 µs ± 52.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.59 ms ± 32.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.92 ms ± 447 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
16.6 ms ± 901 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
16.5 ms ± 571 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


NameError: name 'listas' is not defined